# Homework 4: Preprocessing 

## Introduction

A crucial step when using machine learning algorithms on real-world datasets is preprocessing. This homework will give you some practice of data preprocessing and building a supervised machine learning pipeline on a real-world dataset. 

## Imports 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.dummy import DummyClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings('ignore')

## Exercise A: Introducing the dataset
<hr>

In this lab, you will be working on [the adult census dataset](https://www.kaggle.com/uciml/adult-census-income#). 
Download the CSV and save it as `adult.csv` under this homework folder. 

This is a classification dataset and the classification task is to predict whether income exceeds 50K per year or not based on the census data. You can find more information on the dataset and features [here](http://archive.ics.uci.edu/ml/datasets/Adult).

The starter code below loads the data CSV (assuming that it is saved as `adult.csv` in this folder). 

_Note that many popular datasets have sex as a feature where the possible values are male and female. This representation reflects how the data were collected and is not meant to imply that, for example, gender is binary._

In [ ]:
census_df = pd.read_csv("adult.csv")
census_df.shape

### Data splitting 
rubric={points:5}

In order to avoid violation of the golden rule, the first step before we do anything is splitting the data. 

**Your tasks:**

1. Split the data into `train_df` (80%) and `test_df` (20%) with `random_state = 24`. Keep the target column (`income`) in the splits so that we can use it in the exploratory data analysis.  

_Usually having more data for training is a good idea. But here I'm using 80%/20% split because this is kind of a big dataset for a modest laptop. A smaller training data means it won't take too long to train the model on your laptop. A side advantage of this would be that with a bigger test split, we'll have a more reliable estimate of the deployment performance!_

In [ ]:
train_df = None
test_df = None

# BEGIN YOUR CODE HERE


# END YOUR CODE HERE

## Exercise B: Exploratory data analysis (EDA)
<hr>

Let's examine our `train_df`. 

In [ ]:
train_df.sort_index()

We see some missing values represented with a "?". Probably these were the questions not answered by some people during the census.  Usually `.describe()` or `.info()` methods would give you information on missing values. But here, they won't pick "?" as missing values as they are encoded as strings instead of an actual NaN in Python. So let's replace them with `np.nan` before we carry out EDA. If you do not do it, you'll encounter an error later on when you try to pass this data to a classifier. 

In [ ]:
train_df_nan = train_df.replace("?", np.nan)
test_df_nan = test_df.replace("?", np.nan)
train_df_nan.shape

In [ ]:
train_df_nan.sort_index()

The "?" symbols are now replaced with `NaN` values. 

### Visualizing features
rubric={points:10}

#### Task B1
rubric={points:4}

`display` the information given by `train_df_nan.info()` and `train_df_nan.describe()` methods. 
In the case of `.describe()`, you can **optimally** use the `include="all"` argument to show summary statistics of all  features. 

In [ ]:
# BEGIN YOUR CODE HERE


# END YOUR CODE HERE

#### Task B2 
rubric={points:6}

Visualize the histograms of numeric features 

Hint: use `dataframe.hist` to show the distribution of six numeric features.

In [ ]:
# BEGIN YOUR CODE HERE


# END YOUR CODE HERE

#### Task B3
rubric={points:4}

From the visualizations, which features seem relevant for the given prediction task?(You can pick multi-features).

<font color='red'>YOUR ANSWER HERE</font>

### Identify transformations to apply

rubric={points:20}


#### Task B4
rubric={points:13}

Identify what kind of feature transformations (`scaling`, `imputation`, `one hot encoding`) you would apply on each column in the dataset and fill in the table below accordingly. You may decide to apply any transformations on a certain column or entirely drop a column from your model. That's totally fine. 

As an example, we use imputation and One-Hot encoding for feature `occupation` here.

| Feature | Transformation |
| --- | ----------- |
| occupation | imputation, One-Hot Encoding |
| age | |
| workclass |  |
| fnlwgt | |
| education ||
| education.num |  |
| marital.status |  |
| relationship |  |
| race | |
| sex |  |
| capital.gain | |
| capital.loss |  |
| hours.per.week |  |
| native.country |  |


| Feature | Transformation |
| --- | ----------- |
| occupation | imputation, One-Hot Encoding |
| age | scaling|
| workclass | imputation, One-Hot Encoding |
| fnlwgt | scaling |
| education |One-Hot Encoding|
| education.num | Scaling |
| marital.status | One-Hot Encoding |
| relationship | One-Hot Encoding |
| race | Drop|
| sex | One-Hot Encoding |
| capital.gain | scaling |
| capital.loss | scaling |
| hours.per.week | scaling |
| native.country | imputation, One-Hot Encoding |


#### Task B5
rubric={points:5}

Identify different feature types for applying different transformations. 
In particular, fill in the lists below.

Hint:
1. This dataset is very special - the features with missing values are categorical. So we don't create a list for `imputation_features`.

In [ ]:
# It's OK to keep some of the lists empty or add new lists.
numeric_features = [] 
categorical_features = [] 
ordinal_features = [] 
binary_features = [] 
drop_features = []  # do not include these features in modeling
passthrough_features = [] # do not apply any transformation

# Example: numeric_features = ["age"] 
# BEGIN YOUR CODE HERE


# END YOUR CODE HERE

target = "income"


#### Task B6
rubric={points:2}

Is including the `race` feature for predicting income ethically a good idea? Briefly discuss.

Hint:
1. This question is a bit open-ended and there is no single correct solution.

<font color='red'>YOUR ANSWER HERE</font>

### Separating feature vectors and targets  
rubric={points:6}

<br>

#### Task B7
rubric={points:4}

Create `X_train`, `y_train`, `X_test`, `y_test` from `train_df_nan` and `test_df_nan`.

Hint:
1. `income` is the target.
2. The rest are considered as features.

In [ ]:
# Split the dataset, feature/target:
# BEGIN YOUR CODE HERE


# END YOUR CODE HERE

#### Task B8
rubric={points:2}

At this point, if you train kNN model on `X_train` and `y_train`, would it work? Why or why not?

<font color='red'>YOUR ANSWER HERE</font>

## Exercise C: Preprocessing
<hr>

### Preprocessing using `sklearn`'s `ColumnTransformer` and `Pipeline`
rubric={points:18}

Let's carry out preprocessing using `sklearn`'s `ColumnTransformer` and `Pipeline`. Note that you can define pipelines in two ways: 
- by using [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) and explicitly providing named steps
- by using [`make_pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html#sklearn.pipeline.make_pipeline), which automatically names the steps in the pipeline with their class names. 

Similarly you can create a column transformer in two ways:
- by using [`ColumnTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html)
- by using [`make_column_transformer`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.make_column_transformer.html) 

You may use the method of your choice but `make_pipeline` and `make_column_transformer` are highly recommended.  

#### Task C1 
rubric={points:10}

Create a column transformer `preprocessor` based on transformations you want to apply on the data from [Task B5](#Task-B5).

Hint
1. There are several features with missing values. Fortunately, they are categorical features.
2. Don't forget add `SimpleImputer(strategy='most_frequent')` and `OneHotEncoder` for your `categorical_features`.
3. You can use `make_pipeline` to combine `SimpleImputer` and `OneHotEncoder`.

In [ ]:
preprocessor = None
# BEGIN YOUR CODE HERE


# END YOUR CODE HERE

#### Task C2 
rubric={points:4}

Transform the data by calling `fit_transform` on the training set. 
Then **print** or **display** the shape of the transformed data.

In [ ]:
# BEGIN YOUR CODE HERE


# END YOUR CODE HERE

#### Task C3
rubric={points:4}


Why do we need to use a column transformer in this case? Briefly explain.

<font color='red'>YOUR ANSWER HERE</font>

## Exercise D: Building models
<hr>

Now that we have preprocessed features, we are ready to build models. Below, I'm providing the function we used in class which returns mean cross-validation score along with standard deviation for a given model. Feel free to use it to keep track of your results if you like. 

In [ ]:
results_dict = {} # dictionary to store all the results

In [ ]:

def mean_std_cross_val_scores(model, X_train, y_train, **kwargs):
    """
    Returns mean and std of cross validation

    Parameters
    ----------
    model :
        scikit-learn model
    X_train : numpy array or pandas DataFrame
        X in the training data
    y_train :
        y in the training data

    Returns
    ----------
        pandas Series with mean scores from cross_validation
    """

    scores = cross_validate(model, X_train, y_train, **kwargs)

    mean_scores = pd.DataFrame(scores).mean()
    std_scores = pd.DataFrame(scores).std()
    out_col = []

    for i in range(len(mean_scores)):
        out_col.append((f"%0.3f (+/- %0.3f)" % (mean_scores[i], std_scores[i])))

    return pd.Series(data=out_col, index=mean_scores.index)

### Baseline model 
rubric={points:6}

#### Task D1
rubric={points:3}

Define a **pipeline** with two steps: `preprocessor` from [Task C2](Task-C2) and `scikit-learn`'s `DummyClassifier` with `strategy="prior"` as your classifier.

In [ ]:
# BEGIN YOUR CODE HERE


# END YOUR CODE HERE

#### Task D2
rubric={points:3}

Carry out 5-fold cross-validation with the pipeline. Store the results in `results_dict['dummy]` above. 

> You may use the function `mean_std_cross_val_scores` above to carry out cross-validation and storing results. Refer to the class notes if you are unsure about how to use it. 

In [ ]:
results_dict["dummy"] = None
# BEGIN YOUR CODE HERE


# END YOUR CODE HERE

Now let us show the results.

In [ ]:
pd.DataFrame(results_dict).T

### Trying different classifiers
rubric={points:12}

#### Task D3
rubric={points:5}

For each of the models (`DecisionTreeClassifier` and `KNeighborsClassifier`) in the starter code below:

- Define a pipeline with two steps: `preprocessor` from [Task C2](#Task-C2) and the model as your classifier. 
- Carry out 5-fold cross-validation with the pipeline.  
- Store the results in `results_dict`. 
    

In [ ]:
models = {
    "decision tree": DecisionTreeClassifier(),
    "kNN": KNeighborsClassifier(),
}

In [ ]:
np.random.seed(12345)
# BEGIN YOUR CODE HERE


# END YOUR CODE HERE

#### Task D4
rubric={points:2}

Display all the results so far as a pandas dataframe.

In [ ]:
# BEGIN YOUR CODE HERE


# END YOUR CODE HERE

### Exploring importance of scaling
rubric={points:10}

In this exercise you'll examine whether scaling helps in case of KNNs. 

#### Task D6
rubric={points:4}

Create a column transformer **without** the `StandardScaler` step for `numeric_features`. 
You can refer your to [Task C1](#Task-C1).

In [ ]:
# BEGIN YOUR CODE HERE


# END YOUR CODE HERE

#### Task D7
rubric={points:4}

Repeat the steps in [Task D3](#Task-D3) with this new column transformer. Save all results in `results_dic_compare`.

In [ ]:
np.random.seed(12345)
results_dict_compare = {}  # dictionary to store all the results

# BEGIN YOUR CODE HERE


# END YOUR CODE HERE

In [ ]:
pd.DataFrame(results_dict_compare).T # compare the result with the outcome without doing feature scaling   

#### Task D8
rubric={points:2}

Compare the results of scaled numeric features with unscaled numeric features. 
1. Is scaling necessary for decision trees? (Yes/No)
2. Is scaling necessary for knn? (Yes/No)

<font color='red'>YOUR ANSWER HERE</font>

### Hyperparameter optimization
rubric={points:8}

In this exercise, you'll carry out hyperparameter optimization for the hyperparameter `n_neighbors` of KNeighborsClassifier. 
In practice you'll carry out hyperparameter optimization for all different hyperparameters for the most promising classifiers. 
For the purpose of this assignment, we'll only do it for the K Neighbors classifier with one hyperparameter: `n_neighbors`. 

#### Task D9
rubric={points:4}

For each `n_neighbors` value in the `param_grid` in the starter code below: 
- Create a pipeline object with two steps: `preprocessor` from [Task C1](#Task-C1) and KNeighbors classifier with the value of `n_neignbors`.
- Carry out 5-fold cross validation with the pipeline using the function `mean_std_cross_val_scores`.  
- Store the results in `results_dict_hyper` where the key is the `n_neignbors` value and the value is times and scores.
- This step take a few minutes as you are training 4 knn models with 5 fold cross-validation.
- Display results as a pandas DataFrame.
    

In [ ]:
param_grid = [2, 3, 4, 5]
param_grid

In [ ]:
results_dict_hyper = {}

# BEGIN YOUR CODE HERE


# END YOUR CODE HERE

pd.DataFrame(results_dict_hyper).T

You can find the best hyper-parameter from `[2, 3, 4, 5]`. Keep it in your mind.

## Exercise E: Evaluating on the test set
<hr>

Now that we have a best performing model, it's time to assess our model on the set aside test set. In this exercise you'll examine whether the results you obtained using cross-validation on the train set are consistent with the results on the test set. 

### Task E1
rubric={points:3}

Train the best performing model on **the entire training set** with the pipeline define from [Task D9](#Task-D9).

Hint:
1. Cross validation is no longer needed as you have the best hyper-parameters
2. You build a knn model with `X_train` and `y_train`.

In [ ]:
# BEGIN YOUR CODE HERE


# END YOUR CODE HERE

### Task E2
rubric={points:2}

Report the prediction of this knn model on `X_test`. 

In [ ]:
# BEGIN YOUR CODE HERE


# END YOUR CODE HERE

We can evaluate the trained model with `accuracy_score` which means the accurate prediction rate.

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, X_test_predictions)
print(accuracy)

### Task E3
rubric={points:2}

Are the cross-validation results and test results consistent? (Yes or No)

<font color='red'>YOUR ANSWER HERE</font>

## Submission instructions 

**PLEASE READ:** When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from “1” will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment at Canvas. 
4. Finish the corresponding reflection survey.